In [6]:
import pandas as pd
from decimal import Decimal

from carbon import CarbonSimulatorUI, __version__, __date__
print(f"Carbon Version v{__version__} ({__date__})", )

Carbon Version v2.0-beta3 (12/Dec/2022)


# Carbon Simulation - Demo 5-6

In this demo we test the `ManualRouter` routing method.

In [7]:
ManualSim = CarbonSimulatorUI(pair="ETH/USDC", verbose=False, matching_method='manual', raiseonerror=True)

In [8]:
test_orders = pd.read_csv('256 orders with curr rate alternating.csv', dtype=float)
test_orders.rename(columns={'lowest-rate':'lowest_rate', 'highest-rate':'highest_rate', 'current-rate':'current_rate'}, inplace=True)
test_orders['one_on_lowest_rate'] = 1/test_orders.lowest_rate
test_orders['one_on_highest_rate'] = 1/test_orders.highest_rate
test_orders['one_on_current_rate'] = 1/test_orders.current_rate
test_orders['y_int'] = [test_orders.liquidity[i] * (test_orders.one_on_highest_rate[i] - test_orders.one_on_lowest_rate[i]) / (test_orders.one_on_current_rate[i] - test_orders.one_on_lowest_rate[i]) for i in test_orders.index]
test_orders['fix_trade'] = test_orders.y_int - test_orders.liquidity
test_orders['y_int_minus_fixtrade'] = test_orders.y_int - test_orders.fix_trade
test_orders

,id,liquidity,lowest_rate,highest_rate,current_rate,one_on_lowest_rate,one_on_highest_rate,one_on_current_rate,y_int,fix_trade,y_int_minus_fixtrade
0,0.0,254814732.0,1255.0,2255.0,1256.0,0.000797,0.000443,0.000796,1.419279e+11,1.416730e+11,254814732.0
1,1.0,253827078.0,1254.0,2254.0,2254.0,0.000797,0.000444,0.000444,2.538271e+08,0.000000e+00,253827078.0
2,2.0,252839424.0,1253.0,2253.0,1753.0,0.000798,0.000444,0.000570,3.934554e+08,1.406160e+08,252839424.0
3,3.0,251851770.0,1252.0,2252.0,1253.0,0.000799,0.000444,0.000798,1.401289e+11,1.398770e+11,251851770.0
4,4.0,6913578.0,1004.0,2004.0,1504.0,0.000996,0.000499,0.000665,1.037727e+07,3.463689e+06,6913578.0
...,...,...,...,...,...,...,...,...,...,...,...
251,251.0,250864116.0,1251.0,2251.0,2251.0,0.000799,0.000444,0.000444,2.508641e+08,2.980232e-08,250864116.0
252,252.0,5925924.0,1003.0,2003.0,1004.0,0.000997,0.000499,0.000996,2.970358e+09,2.964432e+09,5925924.0
253,253.0,4938270.0,1002.0,2002.0,2002.0,0.000998,0.000500,0.000500,4.938270e+06,0.000000e+00,4938270.0
254,254.0,3950616.0,1001.0,2001.0,1501.0,0.000999,0.000500,0.000666,5.926911e+06,1.976295e+06,3950616.0


In [9]:
for i in test_orders.index:
    ManualSim.add_strategy(
        tkn='ETH',
        amt_sell=Decimal(test_orders.y_int[i]),
        psell_start=1/Decimal(test_orders.highest_rate[i]),
        psell_end=1/Decimal(test_orders.lowest_rate[i]),
        pbuy_start=None,
        pbuy_end=None,
        pair='ETH/USDC',
        psell_marginal=1/Decimal(test_orders.current_rate[i]),
        pbuy_marginal=None
    )

results = ManualSim.state()['orders']
# results['y_eq'] = results['y_int']==results['y']
# results = results[results['y_eq']==False]
results

,id,pair,tkn,y_int,y,y_unit,disabled,p_start,p_end,p_marg,p_unit,lid
0,0,ETHUSDC,ETH,1.419279e+14,1.419279e+11,ETH,False,0.000443,0.000797,0.000796,USDC per ETH,1
1,1,ETHUSDC,USDC,0.000000e+00,0.000000e+00,USDC,True,None,None,None,USDC per ETH,0
2,2,ETHUSDC,ETH,2.538271e+08,2.538271e+08,ETH,False,0.000444,0.000797,0.000444,USDC per ETH,3
3,3,ETHUSDC,USDC,0.000000e+00,0.000000e+00,USDC,True,None,None,None,USDC per ETH,2
4,4,ETHUSDC,ETH,7.869108e+08,3.934554e+08,ETH,False,0.000444,0.000798,0.000583,USDC per ETH,5
...,...,...,...,...,...,...,...,...,...,...,...,...
507,507,ETHUSDC,USDC,0.000000e+00,0.000000e+00,USDC,True,None,None,None,USDC per ETH,506
508,508,ETHUSDC,ETH,1.185382e+07,5.926911e+06,ETH,False,0.0005,0.000999,0.000686,USDC per ETH,509
509,509,ETHUSDC,USDC,0.000000e+00,0.000000e+00,USDC,True,None,None,None,USDC per ETH,508
510,510,ETHUSDC,ETH,1.482962e+12,1.482962e+09,ETH,False,0.0005,0.001,0.000999,USDC per ETH,511


In [10]:
ttl_liquidity = results['y'].sum()

def harmonic_sum(n):
    i = 1
    s = 0.0
    routes = []
    for i in range(1, 10):
        if i % 2 == 0:
            print(i, Decimal(1/i)*100)
            routes.append((i, Decimal(1/i)*100))
            s = s + 1/i*100
    print('total amt', Decimal(s))
    return Decimal(s), routes

n = 10
amt, routes = harmonic_sum(n)
ManualSim.amm_sells('ETH', amt, execute=False, use_routes=routes)['trades']

2 50.0
4 25.00
6 16.6666666666666657414808128123695496469736099243164062500
8 12.500
total amt 104.166666666666657192763523198664188385009765625


,uid,id,subid,note,aggr,exec,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,0.0,0,0,route #2,False,False,None,50.000000,ETH,4.092002e+17,USDC,ETHUSDC,2,1,10378099289566760.0,USDC per ETH
0,0.1,0,1,route #4,False,False,None,25.000000,ETH,2.686538e+17,USDC,ETHUSDC,4,1,10378099289566760.0,USDC per ETH
0,0.2,0,2,route #6,False,False,None,16.666667,ETH,2.453962e+17,USDC,ETHUSDC,6,1,10378099289566760.0,USDC per ETH
0,0.3,0,3,route #8,False,False,None,12.500000,ETH,1.578018e+17,USDC,ETHUSDC,8,1,10378099289566760.0,USDC per ETH
0,0,0,A,AMM sells 104ETH buys 1081052009329874176USDC,True,False,None,104.166667,ETH,1.081052e+18,USDC,ETHUSDC,"[2, 4, 6, 8]",4,1.037809928956676e+16,USDC per ETH
